Import Packages

In [46]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=MfyX2wRTy8pSpGtjn-8FNvzmi5zugWt5O6_heCPz4hY&tc=DuDJoLrJXCvbpSMPRhdmT6MB4Y7EoNzBvyPC3Ll-2MU&cc=pDpeFZUUDZqNOzxy8GGKPCM9OShSzXpuXQT1ccaEvAk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnBTcxA4JaHIZ9hxpeUSFgCl9tpjHXnwSJ5JfdqtX2-NaPLbqwN3X4

Successfully saved authorization token.


In [47]:
import ee
import geemap.core as geemap

In [48]:
Landsat8 = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_170052_20170108')\
.select('B4', 'B3', 'B2')

#CREATE REGION
region = ee.Geometry.Rectangle(37.07, 11.50, 37.39, 11.82)

In [49]:
#Export to Google Drive
task = ee.batch.Export.image.toDrive(**{
    'image': Landsat8,
    'description': 'imagetoDriveExample',
    'folder': 'Example_folder',
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

Create a Collection

In [54]:
geometry = ee.Geometry.Point([31.5051, -26.6061])

#Location = roi.geometry().centroid().coordinates().getInfo()[::-1]
location = geometry.coordinates().getInfo()[::-1]

s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
rgbVis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])


filtered = s2.filter(ee.Filter.date('2020-01-01', '2020-05-01')) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
    .filter(ee.Filter.bounds(geometry)) \
    .map(maskS2clouds)

#Write a function for NDVI for an image and adds it as a band
def addNDVI (image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('ndvi')
  return image.addBands(ndvi)

withNdvi = filtered.map(addNDVI)
median = withNdvi.median()



In [55]:
import folium

#Define a method for displaying EE image tiles to folium map
def addLayer (self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Map Data &copy; <a href=https://earthengine.google.com/">Google Earth Engine<a/>',
      name = name,
      overlay = True,
      control = True,
  ).add_to(self)

#Add EE drawing method to folium
folium.Map.addLayer = addLayer

#Create the map object
Map = folium.Map(location=location, zoom_start = 10)

Map.addLayer(median, rgbVis, 'Sentinel')
Map.add_child(folium.LayerControl())
display(Map)

In [52]:
image_ids = withNdvi.aggregate_array('system:index').getInfo()
print('Total images: ', len(image_ids))

Total images:  11


In [53]:
#Export with 100m resolution for this demo
for i, image_id in enumerate(image_ids):
  image = ee.Image(withNdvi.filter(ee.Filter.eq('system:index', image_id)).first())
  task = ee.batch.Export.image.toDrive(**{
      'image': image.select('ndvi'),
      'description': 'Image Export {}'.format(i+1),
      'fileNamePrefix': image.id().getInfo(),
      'folder': 'Example_folder',
      'scale': 100,
      'region': image.geometry().bounds().getInfo()['coordinates'],
      'maxPixels': 1e10
  })
  task.start()
  print('Started Task: ', i+1)

Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7
Started Task:  8


Started Task:  9
Started Task:  10
Started Task:  11
